In [ ]:
""" 
# Procedimentos a serem realizados
# 1 - Criação de uma chave Universal para realizar a verificação de registros duplicados
#    1.1 - Converter as colunas: ['data_emprestimo'], ['data_devolucao'] e ['data_renovacao'] para Datetime
#    1.2 - Criar a coluna ['DataEmprestimo_fmt'] para armazenar a Data e Hora da coluna ['data_emprestimo'] no formato: yyyymmddhhmmss
#    1.3 - Retirar a casa decimal da coluna ['matricula_ou_siape']
#       a - Converte a coluna para STRING
#       b - Eliminar os 2 últimos caracteres [.0]
#       c - Criar uma nova coluna ['ID_MatriculaSiape'] para armazenar o novo valor
#   1.4 - Converter todos os caracteres que não forem números para números da coluna ['codigo_barras']
#   1.5 - Armazenar na coluna ['ID_Unico'] os dados concatenados das colunas:
#       ['id_emprestimo'].astype(str) + ['ID_CodigoBarras'].astype(str) + df_Livros['ID_MatriculaSiape'] + df_Livros['DataEmprestimo_fmt']
"""
import pandas as pd

def converter_nao_numericos_para_ascii(texto):
  """ Converte caracteres não numéricos em um texto para ASCII e mantém os numéricos. """
  resultado = []
  for caractere in texto:
    if caractere.isdigit():
      resultado.append(caractere)
    else:
      resultado.append(str(ord(caractere)))
  return int(''.join(resultado))  

pd.options.mode.copy_on_write = True
 
_strCaminho = 'D:/Users/rtoni/OneDrive/Git-Dados/Projetos/7DaysOfCode.io/Dados/'
_strDB_Livros = 'DB_Emprestimo.csv'
_strDB_Parquet = 'DB_CadLivros.csv'
df_Livros = pd.read_csv(f'{_strCaminho}{_strDB_Livros}', sep=';')
df_Parquet = pd.read_csv(f'{_strCaminho}{_strDB_Parquet}', sep=';')
# 
df_Livros['data_emprestimo'] = pd.to_datetime(df_Livros['data_emprestimo'])
df_Livros['data_devolucao']  = pd.to_datetime(df_Livros['data_devolucao'])
df_Livros['data_renovacao']  = pd.to_datetime(df_Livros['data_renovacao'])
#
df_Livros['DataEmprestimo_fmt'] = df_Livros['data_emprestimo'].dt.strftime('%Y%m%d%H%M')
#
df_Livros['codigo_barras']   = df_Livros['codigo_barras'].str.strip()
df_Livros['ID_CodigoBarras'] = df_Livros['codigo_barras'].apply(converter_nao_numericos_para_ascii)
#
df_Livros['matricula_ou_siape'] = df_Livros['matricula_ou_siape'].astype(str)
df_Livros['matricula_ou_siape'] = df_Livros['matricula_ou_siape'].str.strip()
df_Livros['ID_MatriculaSiape']  = df_Livros['matricula_ou_siape'].str.slice(stop=-2)
#
df_Livros['Id_Unico'] = ( 
    df_Livros['id_emprestimo'].astype(str) + 
    df_Livros['ID_CodigoBarras'].astype(str) +
    df_Livros['ID_MatriculaSiape'] + 
    df_Livros['DataEmprestimo_fmt']
)
#
df_Livros.drop(['DataEmprestimo_fmt', 'ID_CodigoBarras', 'ID_MatriculaSiape'], axis=1, inplace=True)
#
df_Livros.info()
df_Livros.head()

In [ ]:
df_Livros

In [ ]:
""" 
# Faz a verificação e a contagem dos registros duplicados
# Foram encontradas 37 duplicidades no total
# Apagados os registros em duplicidade
"""
df_Livros.duplicated().sum()

In [ ]:
df_Livros['ID_Unico'].value_counts()

In [ ]:

df_Livros = df_Livros.drop_duplicates(subset=['ID_Unico'], keep='first')
#


In [ ]:
# df_Livros['ID_CodigoBarras'] = df_Livros['codigo_barras'].apply(lambda x: int(''.join(str(ord(c)) for c in x)))


# _intCont = 0
# for elemento in df_Livros['id_emprestimo']:
#     #
#     if not df_Livros['matricula_ou_siape'][_intCont] == 'matricula_ou_siape':
#         #
#         _intQtdCarac   = 0
#         _strMatricula  = ''
#         _strID_Unico   = '' 
#         _strDtHrSerial = ''
#         #
#         _intQtdCarac  = len(df_Livros['matricula_ou_siape'][_intCont]) - 2
#         _strMatricula = df_Livros['matricula_ou_siape'][_intCont][:_intQtdCarac]
#         #
#         _strDtHrSerial =  df_Livros['data_emprestimo'][_intCont][:4]       # Ano
#         _strDtHrSerial += df_Livros['data_emprestimo'][_intCont][5:7]      # Mes
#         _strDtHrSerial += df_Livros['data_emprestimo'][_intCont][8:10]     # Dia
#         _strDtHrSerial += df_Livros['data_emprestimo'][_intCont][11:13]    # Hora
#         _strDtHrSerial += df_Livros['data_emprestimo'][_intCont][14:16]    # Minuto
#         _strDtHrSerial += df_Livros['data_emprestimo'][_intCont][17:19]    # Segundo
#         """         
#         # Monta o ID Único: 
#         # _strID_Unico = ['id_emprestimo'].astype(str) + ['codigo_barras'] + ['matricula_ou_siape'] + _strDataHoraSerial
#         # Ao armazenar na coluna ['Id_Unico'] é convertido para INT
#         """        
#         _strID_Unico = str(df_Livros['id_emprestimo'][_intCont]) + df_Livros['codigo_barras'][_intCont] + _strMatricula + _strDtHrSerial
#         df_Livros['Id_Unico'][_intCont] = _strID_Unico
#     else:
#         df_Livros['matricula_ou_siape'][_intCont] = '0'
#     #
#     _intCont += 1